![](../images/rivacon_frontmark_combined_header.png)

In [ ]:
from IPython.display import HTML

HTML('''<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>''')

In [ ]:
import copy # needed for deppcopy of dictionaries
import pyvacon.analytics as analytics
import pyvacon.tools.enums as enums
import pyvacon.tools.converter as converter
import matplotlib.pyplot as plt
import numpy as np
import pylab as pl
import pandas as pd
from ipywidgets import widgets
from IPython.display import display, clear_output, Image
from pylab import rcParams
%matplotlib notebook
#!ipython profile locate default
import datetime

### Settings

In [ ]:

refdate = datetime.datetime(2018,4,1)
#selected_spec.value = 'rivacon_injection_nonvolumedependent' #'boogert_de_jong'
selected_spec = widgets.Dropdown(
    options=['boogert_de_jong', 'rivacon', 'rivacon_injection_nonvolumedependent', 'boogert_de_jong_time_dependent'],
    value='rivacon',
    #description='Number:',
    #disabled=False,
    button_style='' # 'success', 'info', 'warning', 'danger' or ''
)

selected_spec_new = widgets.Dropdown(
    options=['', 'boogert_de_jong', 'rivacon', 'rivacon_injection_nonvolumedependent', 'boogert_de_jong_time_dependent'],
    value='',
    description='Reference spec to compare:',
    #disabled=False,
    button_style='' # 'success', 'info', 'warning', 'danger' or ''
)


# type of experiments
readFromCsv = widgets.Checkbox(
    #value=False,
    description='Forward Curve from CSV',
)

#widgets.interact(readFromCsv)
bid_ask_dispatch = widgets.Checkbox(
    value=False,
    description='Bid/Ask dispatch'
)
bid_ask_projection = widgets.Checkbox(
    value=False,
    description='Bid/Ask projection'
)
volume_dependent_injection_cost_dispatch = widgets.Checkbox(
    value=False,
    description='Injection Costs dispatch'
)
volume_dependent_injection_cost_projection = widgets.Checkbox(
    value=False,
    description='Injection Costs projection'
)
volume_independent_injection_rate_projection =widgets.Checkbox(
    value=False,
    description='Volume independent Injection Rates projection'
)
volume_dependent_injection_rate_projection = widgets.Checkbox(
    value=False,
    description='Volume dependent Injection Rates projection'
)
display(selected_spec, selected_spec_new, readFromCsv, bid_ask_dispatch, bid_ask_projection, volume_dependent_injection_cost_dispatch, volume_dependent_injection_cost_projection,
      volume_independent_injection_rate_projection,  volume_dependent_injection_rate_projection)


#intrinsic settings
volume_points = 50
num_injections = 10
#lsmc settings
pathgenerator_param = analytics.PathGeneratorParameter()
pathgenerator_param.numberOfSimulations = 1000
pathgenerator_param.numberOfTimeStepsPerYear = 365
pathgenerator_param.numberOfCheckpoints = 5
pathgenerator_param.maxNumThreads = 1


#internals
valdate = converter.getLTime(refdate)



### Specification

In [ ]:
MWh_to_m3_factor = 1
specs = {
'boogert_de_jong': {'description': ' Create a specification as in Gas Storage Valuation Using a Monte Carlo Method from Boogert, de Jong, Wokring Paper 2006'
                 , 'ttm' : 365, 'min_vol' : 0, 'max_vol': 250000.0*MWh_to_m3_factor,
                 'start_volume' : 100000*MWh_to_m3_factor,
                 #'withdrawal_max' : -7500*MWh_to_m3_factor,
                 #'injection_max' : 2500*MWh_to_m3_factor,
                  'injection_cost_volumes': [0, 1000000000],
                  'injection_cost': [0,0],
                    'injection_rate_volumes':[0,100000000],
                    'injection_rate': [2500*MWh_to_m3_factor, 2500*MWh_to_m3_factor],
                  'withdrawal_cost_volumes': [-1000000000,0],  
                'withdrawal_rate_volumes':[0,100000000],
                    'withdrawal_rate': [ -7500*MWh_to_m3_factor,  -7500*MWh_to_m3_factor],
                  'withdrawal_cost' : [0, 0],
                  'final_volume' : 100000*MWh_to_m3_factor,
                  'start' : refdate + datetime.timedelta(days=1),
                  'granularity' : 24},
'rivacon': {'description': ' Create a specification as in Gas Storage Valuation'
                 , 'ttm' : 365, 'min_vol' : 0.0, 'max_vol': 60000.0, # 60GWh => 60000/24=2500 MWdays
                  'start_volume' : 0.0,
                  'injection_cost_volumes': [0.0, 1000000000.0],
                  'injection_cost': [0.4, 0.4],
                  'injection_rate_volumes':[0.0, 0.999*36000.0, 36000.0,  0.999*48000.0,48000.0, 100000000.0],
                  'injection_rate': [24.0*30.0, 24.0*30.0, 24.0*21.0, 24.0*21.0, 24.0*9.0, 24.0*9.0], # it takes 144 days of injection to reach max volume
                  'withdrawal_cost_volumes': [-1000000000.0, 0.0],
                  'withdrawal_cost' : [0.4, 0.4],
                  'withdrawal_rate_volumes':[0.0, 100000000.0],
                  'withdrawal_rate': [ -24.0*30.0, -24.0*30.0], # it takes 84 days to empty the storage
                  'final_volume' : 0.0,
                  'start' : refdate + datetime.timedelta(days=1),
                  'granularity' : 24,
                   'final_penalty':{'volumes' :[0,1.0, 2.0, 10000000000], 'values':[0,0,-1000000000000, -1000000000000]}},
'rivacon_injection_nonvolumedependent': {'description': ' Create a specification as in Gas Storage Valuation'
                 , 'ttm' : 365, 'min_vol' : 0.0, 'max_vol': 60000.0, # 60GWh => 60000/24=2500 MWdays
                  'start_volume' : 0.0,
                  'injection_cost_volumes': [0.0, 1000000000.0],
                  'injection_cost': [0.4, 0.4],
                  'injection_rate_volumes':[0.0, 36000.0, 48000.0, 100000000.0],
                  'injection_rate': [24.0*30.0,24.0*30.0,24.0*30.0,24.0*30.0],  # it takes 144 days of injection to reach max volume
                  'withdrawal_cost_volumes': [-1000000000.0, 0.0],
                  'withdrawal_cost' : [0.4, 0.4],
                  'withdrawal_rate_volumes':[0.0, 100000000.0],
                  'withdrawal_rate': [ -24.0*30.0, -24.0*30.0], # it takes 84 days to empty the storage
                  'final_volume' : 0.0,
                  'start' : refdate + datetime.timedelta(days=1),
                  'granularity' : 24,
                   'final_penalty':{'volumes' :[0,1.0, 2.0, 10000000000], 'values':[0,0,-1000000000000, -1000000000000]}},
'boogert_de_jong_time_dependent': {
        'description': ' Create a specification as in Gas Storage Valuation Using a Monte Carlo Method from Boogert, de Jong, Wokring Paper 2006'
                 , 'ttm' : 365, 
                 'start_volume' : 100000*MWh_to_m3_factor,
                
                'operation_details': [ {
                    'start_date': datetime.datetime(2010,1,1), 
                        'min_vol' : 0, 'max_vol': 250000.0*MWh_to_m3_factor,
                        'injection':{
                                'volumes': [0, 1000000000],
                                'costs': [0,0],
                                'rates': [2500*MWh_to_m3_factor, 2500*MWh_to_m3_factor]
                                    },
                        'withdrawal': {
                                'volumes': [-1000000000,0],  
                                'rates': [ -7500*MWh_to_m3_factor,  -7500*MWh_to_m3_factor],
                                'costs' : [0, 0]
                            }
                    },
                     {
                    'start_date': refdate + datetime.timedelta(days=180),
                        'min_vol' : 0, 'max_vol': 125000.0*MWh_to_m3_factor,
                        'injection':{
                                'volumes': [0, 1000000000],
                                'costs': [0,0],
                                'rates': [2500*MWh_to_m3_factor, 2500*MWh_to_m3_factor]
                                    },
                        'withdrawal': {
                                'volumes': [-1000000000,0],  
                                'rates': [ -7500*MWh_to_m3_factor,  -7500*MWh_to_m3_factor],
                                'costs' : [0, 0]
                            }
                    }
                    ],
                  'final_volume' : 100000*MWh_to_m3_factor,
                  'start' : refdate + datetime.timedelta(days=1),
                  'granularity' : 24
                }
}


### Setup Model

In [ ]:
refdate_p = converter.getLTime(refdate)
current_spot = 1.0
times = [0.0,2.0]
#mr_rates = [0.05*365, 0.05*365]
mr_rates = [10.0, 10.0]
mr_levels = [np.log(current_spot),np.log(current_spot)]
#vols = [0.0945*np.sqrt(365.0), 0.0945*np.sqrt(365.0)] #[0.0945*np.sqrt(365.0), 0.0945*np.sqrt(365.0)]
vols = [0.3, 0.3]

model = analytics.ExponentialOrnsteinUhlenbeck('DummyId', refdate_p, current_spot, times, mr_rates, times, mr_levels, times, vols, False)
#model_lab = analytics.ModelLab(model,refdate_p)
#sim_times = analytics.vectorPTime([analytics.ptime(2018,1,1, 0,0,0), analytics.ptime(2018,2,1, 0,0,0), analytics.ptime(2018,3,1, 0,0,0)])
#model_lab.simulate(sim_times, 100, 365)
#tmp = analytics.vectorDouble()
#model_lab.getPath(tmp, 0, 1)
#path = []
#for i in range(len(tmp)):
#    path.append(tmp[i])
#print(path)
#plt.figure()
#plt.plot(path,'x')
#help(analytics.ExponentialOrnsteinUhlenbeck(refdate, 100, [1.0, 2.0], ))

#### Create selected specification

In [ ]:
def set_injection_descriptions(injection_rate_volumes, injection_rate, 
                                        injection_cost_volumes, injection_cost):
    injection = analytics.InjectionDescription()
    injection.costVolumes = analytics.vectorDouble(injection_cost_volumes)
    injection.cost = analytics.vectorDouble(injection_cost)
    injection.maxVolumes = analytics.vectorDouble(injection_rate_volumes) #constant injection rate independent of storage volume
    injection.minVolumes = injection.maxVolumes
    tmp = []
    for x in injection_rate_volumes:
        tmp.append(0.0)
    if injection_rate[0] >= 0:
        injection.max = analytics.vectorDouble(injection_rate)
        
        injection.min = analytics.vectorDouble(tmp)
    else:
        injection.min = analytics.vectorDouble(injection_rate)
        injection.max = analytics.vectorDouble(tmp)
    return injection

def create_final_penalty(desc):
    finalPenaltyPoints = analytics.vectorDouble([0, 0.9*desc['final_volume'], desc['final_volume'], 1.1*desc['final_volume'], desc['final_volume']*10])
    finalPenaltyValues = analytics.vectorDouble([-1000000000, 0, 0, 0, -1000000000])
    if 'final_penalty' in desc.keys():
        finalPenaltyPoints = desc['final_penalty']['volumes']
        finalPenaltyValues = desc['final_penalty']['values']
    return (finalPenaltyPoints, finalPenaltyValues)
        
#def create_spec_time_dependent(desc):
def create_operation_details(desc):
    operation_details = []
    if 'operation_details' in desc.keys():
        op_details = desc['operation_details']
        for x in op_details:
            injection = x['injection']
            withdrawal = x['withdrawal']
            detail = {
                'start_date': x['start_date'],
                    'min_vol' : x['min_vol'],
                 'max_vol': x['max_vol'],
                 'injection': set_injection_descriptions(injection['volumes'], injection['rates'],
                                                    injection['volumes'], injection['costs']),
                 'withdrawal': set_injection_descriptions(withdrawal['volumes'], withdrawal['rates'],
                                                    withdrawal['volumes'], withdrawal['costs'])
                 }
            operation_details.append(detail)
    else:
        detail = {
                'start_date': datetime.datetime(2100,1,1),
                'min_vol' : desc['min_vol'],
                 'max_vol': desc['max_vol'],
                 'injection': set_injection_descriptions(desc['injection_rate_volumes'], desc['injection_rate'],
                                                    desc['injection_cost_volumes'], desc['injection_cost']),
                 'withdrawal': set_injection_descriptions(desc['withdrawal_rate_volumes'], desc['withdrawal_rate'],
                                                    desc['withdrawal_cost_volumes'], desc['withdrawal_cost'])
                 }
        operation_details.append(detail)
    return operation_details
    
def create_spec(desc):
    final_penalty = create_final_penalty(desc)
    start = converter.getLTime(desc['start'])
    end = analytics.ptime()
    start.addDays(end, desc['ttm'])
    #if isinstance(desc['withdrawal_max'],
    operation_details = create_operation_details(desc)
    gasstorage = analytics.GasStorageSpecification('id', 'issuer', enums.SecuritizationLevel.NONE, 'EUR',  'udl',
                                            operation_details[0]['min_vol'], operation_details[0]['max_vol'], 
                                             desc['start_volume'], start, end, 
                                             operation_details[0]['injection'],
                                              operation_details[0]['withdrawal'], 
                                             desc['granularity'],
                                            desc['final_volume'], final_penalty[0], final_penalty[1])
    for i in range(1,len(operation_details)):
        start_date = converter.getLTime(operation_details[i]['start_date'])
        gasstorage.append(start_date, operation_details[i]['min_vol'], operation_details[i]['max_vol'], 
                           operation_details[i]['injection'], operation_details[i]['withdrawal'])
    return gasstorage

spec = copy.deepcopy(specs[selected_spec.value])
gasstorage = create_spec(spec)

### Create forward curve

In [ ]:
dates = []
fwd = []
rcParams['figure.figsize'] = 12, 3

if readFromCsv.value == True:
    raw_fwd = pd.read_csv('.\Fictional Gas Curve.csv',sep=';')
    for i, d in enumerate(raw_fwd.Date):
        dd = datetime.datetime.strptime(d,'%d.%m.%Y')
        if dd >= refdate:
            dates.append(dd)
            fwd.append(raw_fwd.Value[i])
else:
    random_factor = 0.0
    for i in range(0,365):
        fwd.append(20+ 5*(np.cos(2*np.pi*(i+90)/365)+2) + random_factor*np.random.uniform(-1,5))
        dates.append(refdate + datetime.timedelta(days=i))
plt.figure()
p_dates = converter.createPTimeList(refdate_p, dates)
fwd_curve = analytics.DatedCurve('test', p_dates[0], p_dates, fwd, enums.DayCounter.ACT365_FIXED, 'LINEAR', 'CONSTANT')
plt.plot(dates,fwd,'-x')
plt.ylabel('EUR/MWh')
plt.gcf().autofmt_xdate()
plt.savefig('ForwardCurve.png', dpi=1000)


### Bid/Ask

In [ ]:
def bidask_function(spread, factor, volumes):
    result = []
    for x in volumes:
        result.append(1.0 + spread*np.tanh(factor*x)/np.tanh(factor*np.abs(volumes[0])))
    return result


def create_bid_ask():
   
    bidask = {}
    
    rivacon_example = False
    if selected_spec.value =='rivacon':
        rivacon_example = True
        
    if rivacon_example == True:
        volume_depp = pl.frange(-800, 800, 20.0) #
        volume = []
        for x in volume_depp:
            volume.append(x)
        bidask_2 = []
        for x in volume_depp:
            bidask_2.append(1.0 + 0.02*np.tanh(0.001*x)/np.tanh(0.001*volume[-1]))
        bidask['case 1'] = {'volumes': volume, 'multiplier': bidask_2}

        bidask_3 = []
        for x in volume_depp:
            bidask_3.append(1.0 + 0.035*np.tanh(0.001*x)/np.tanh(0.001*volume[-1]))
        bidask['case 2'] = {'volumes': volume, 'multiplier': bidask_3}

        bidask_4 = []
        for x in volume_depp:
            bidask_4.append(1.0 + 0.035*np.tanh(0.005*x)/np.tanh(0.005*volume[-1]))
        bidask['case 3'] = {'volumes': volume, 'multiplier': bidask_4}
    else:
        volume_depp =  pl.frange(-7500,2500, 100.0) # pl.frange(-800, 800, 20.0) #
        volume = []
        for x in volume_depp:
            volume.append(x)
        bidask_2 = []
        for x in volume_depp:
            bidask_2.append(1.0 + 0.02*np.tanh(0.0001*x)/np.tanh(0.0001*volume[-1]))
        bidask['case 1'] = {'volumes': volume, 'multiplier': bidask_2}

        bidask_3 = []
        for x in volume_depp:
            bidask_3.append(1.0 + 0.0275*np.tanh(0.00035*x)/np.tanh(0.00035*volume[-1]))
        bidask['case 2'] = {'volumes': volume, 'multiplier': bidask_3}

        bidask_4 = []
        for x in volume_depp:
            bidask_4.append(1.0 + 0.035*np.tanh(0.0007*x)/np.tanh(0.0007*volume[-1]))
        bidask['case 3'] = {'volumes': volume, 'multiplier': bidask_4}
    bidask_1 = []
    for x in volume_depp:
        bidask_1.append(1.0 )
    bidask['none'] = {'volumes': volume, 'multiplier': bidask_1}
        
    return bidask

In [ ]:
bidask_old = {'none': {'volumes' : [-100, -40, -20, -0.01, 0, .01, 20, 40, 100], 
                   'multiplier':  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]},
         #'case 1': {'volumes' : [-80, -40, -20, -0.01, 0, .01, 20, 40, 80], 
         #          'multiplier':  [0.99, 0.99, 0.99, 0.99, 1.0, 1.01, 1.01, 1.01, 1.01]},
         'case 1': {'volumes' : [-80, -40, -20, -0.01, 0, .01, 20, 40, 80], 
                   'multiplier':  [0.99, 0.99, 0.99, 1.0, 1.0, 1.0, 1.01, 1.01, 1.01]},
         'case 2':  {'volumes' : [-80, -40, -20, -0.01, 0, .01, 20, 40, 80], 
                   'multiplier':  [0.95, 0.97, 0.99, 1.0, 1.0, 1.0, 1.01, 1.03, 1.05]}
         }
bidask = create_bid_ask()

analytics.setLogLevel('ERROR')

def plot_bidask(bidask_dict):
    plt.figure()
    #plt.title('bid-ask multiplier depending on traded volume')
    for key, values in  bidask_dict.items():
        plt.plot(values['volumes'], values['multiplier'], '-x', label=key)
    plt.xlabel('traded volume [MWh]')
    plt.ylabel('multiplier')
    plt.legend(loc='lower right',prop={'size':10})
    plt.gcf().autofmt_xdate()
    plt.savefig('bidask_spreads.png', dpi = 600)

transaction_volume = [-10, 0, 10]
transaction_cost = [10.0, 10.0, 10.0]


def compute_bidask_intrinsic(result, bidask_dict, lsmc = False):
    for key, values in bidask_dict.items():
        res = {}
        dispatch = analytics.vectorDouble()
        bidask_volumes = analytics.vectorDouble(values['volumes'])
        bidask_multiplier = analytics.vectorDouble(values['multiplier'])
        price = analytics.GasStorageIntrinsicPricer.priceIntrinsic(dispatch, valdate, gasstorage,
                                                         fwd_curve,  volume_points, num_injections, 
                                                         bidask_volumes, bidask_multiplier, 
                                                        transaction_volume, transaction_cost)
        res['intrinsic_price'] = price
        res['dispatch'] = dispatch
        volumes = [spec['start_volume']]
        max_vol = 1
        if 'max_vol' in spec.keys():
            max_vol = spec['max_vol']
        else:
            max_vol = spec['operation_details'][0]['max_vol']
        for x in dispatch:
            last = volumes[-1]
            volumes.append((last+x))
        for i in range(len(volumes)):
            volumes[i] = 100.0*volumes[i] / max_vol
        res['volumes'] = volumes
        if lsmc == True:
            lsmc_dispatch = analytics.vectorDouble()
            lsmc_price = analytics.GasStorageLSMCPricer.price(lsmc_dispatch, valdate, gasstorage, model, fwd_curve, 
                                           bidask_volumes, bidask_multiplier,  
                                           transaction_volume, transaction_cost, 
                                           pathgenerator_param, 50)
            volumes = [spec['start_volume']]
            for x in lsmc_dispatch:
                last = volumes[-1]
                volumes.append((last+x))
            for i in range(len(volumes)):
                volumes[i] = 100.0*volumes[i] / max_vol
            res['lsmc_price'] = lsmc_price
            res['lsmc_dispatch'] = lsmc_dispatch
            res['lsmc_volumes'] = volumes
        result[key] = res

marker_dict = [['none','-x'], ['case 1', '-o'], ['case 2', '-v'], ['case 4', '-*']]


#plt.title('transaction cost vs traded volume')
#plt.plot(transaction_volume, transaction_cost,'-x')
#transaction_volume = analytics.vectorDouble(transaction_volume)
#transaction_cost = analytics.vectorDouble(transaction_cost)
#
if bid_ask_dispatch.value:
    result = {}
    compute_bidask_intrinsic(result, bidask, True)
    rcParams['figure.figsize'] = 6, 4
    plot_bidask(bidask)
    rcParams['figure.figsize'] = 12, 4
    plot_dispatch(result,  'bidask')
    for key, value in result.items():
        print(key + ':  intrinsic_price: ' + str(value['intrinsic_price']) + '  lsmc_price: ' + str(value['lsmc_price']))

#### Price Projection

In [ ]:

analytics.setLogLevel('ERROR')
def plot_bidask_price(result_dict):
    plt.figure()
    plt.subplot(1,2,1)
    for key1 in result_dict:
        result = result_dict[key1]
        #print(result)
        bidask = []
        intrinsic_price = []
        lsmc_price = []
        for key in sorted(result):
            bidask.append(key)
            intrinsic_price.append((result[key])['intrinsic_price'])
        plt.plot(bidask, intrinsic_price, '-x', label = key1)
        plt.xlabel('factor')
        plt.ylabel('value')
        plt.legend(loc='upper right', prop={'size':10})
    plt.subplot(1,2,2)
    for key1 in result_dict:
        result = result_dict[key1]
        #print(result)
        bidask = []
        intrinsic_price = []
        lsmc_price = []
        for key in sorted(result):
            bidask.append(key)
            lsmc_price.append((result[key])['lsmc_price'])
        plt.plot(bidask, lsmc_price, '-x', label = key1) 
        plt.xlabel('factor')
        plt.ylabel('value [EUR]')
        plt.legend(loc='upper right', prop={'size':10})
    plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
    plt.savefig('bidask_prices_proj.png', dpi = 600)
    
spreads = [0.02, 0.04, 0.06, 0.08] 
factors = pl.frange(0.00001,0.002, 0.00025)
bidask_proj = {}
#print(volumes)
for spread in spreads:
    bidask_proj[str(spread)] = {}
    for factor in factors:
        ba = {}
        ba['volumes'] = bidask['none']['volumes']
        ba['multiplier'] = bidask_function(spread, factor, ba['volumes']) #[2.0-x, 2.0-x, 1.0, x, x]
        bidask_proj[str(spread)][factor] = ba

result = {}
if bid_ask_projection.value:
    result = {}
    for spread in spreads:
        key = str(spread)
        result[key] = {}
        compute_bidask_intrinsic(result[key], bidask_proj[key], True)
    plot_bidask_price(result)
    
    plt.figure()
    for factor in factors:
        multi = bidask_function(0.08, factor, bidask['none']['volumes'])
        plt.plot(bidask['none']['volumes'], multi, '-', label=str(factor))
        plt.xlabel('traded volume [MWh]')
        plt.ylabel('bid-ask multiplier')
        plt.legend(loc='upper right', prop={'size':10})
    plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
    plt.savefig('bidask_multiplier_proj.png', dpi = 600)

### Injection/Withdrawal Costs

#### Effect of volume dependent injection costs

In [ ]:
injections = {'none': {'injection_cost_volumes': [0, 250000.0*MWh_to_m3_factor],
                    'injection_cost': [0,0]},
            'case 1': {'injection_cost_volumes': [0, 0.7*250000.0*MWh_to_m3_factor,0.71*250000.0*MWh_to_m3_factor, 250000.0*MWh_to_m3_factor],
                    'injection_cost':[5.0,5.0, 10.0, 10.0]},
            'case 2': {'injection_cost_volumes': [0, 0.4*250000.0*MWh_to_m3_factor,0.41*250000.0*MWh_to_m3_factor, 250000.0*MWh_to_m3_factor],
                    'injection_cost': [5.0, 5.0, 10.0, 10.0]}
           }

def plot_injection_cost(injection_dict):
    plt.figure()
    #plt.title('bid-ask multiplier depending on traded volume')
    for key, values in  injection_dict.items():
        plt.plot(values['injection_cost_volumes'], values['injection_cost'], '-x', label=key)
    plt.xlabel('current volume')
    plt.ylabel('injection cost')
    plt.legend()
    
    
def compute_injection_intrinsic(result, injection_dict, lsmc = False):
    for key, values in injection_dict.items():
        res = {}
        spec_new = copy.deepcopy(specs[selected_spec.value])
        spec_new['injection_cost_volumes'] = values['injection_cost_volumes']
        spec_new['injection_cost'] = values['injection_cost']
        spec_new = create_spec(spec_new)
        dispatch = analytics.vectorDouble()
        price = analytics.GasStorageIntrinsicPricer.priceIntrinsic(dispatch, valdate, spec_new,
                                                     fwd_curve,  volume_points, num_injections, 
                                                     bidask['none']['volumes'], 
                                                    bidask['none']['multiplier'], 
                                                    transaction_volume, transaction_cost)
        res['intrinsic_price'] = price
        res['dispatch'] = dispatch
       
        volumes = [spec['start_volume']]
        for x in dispatch:
            last = volumes[-1]
            volumes.append((last+x))
        for i in range(len(volumes)):
            volumes[i] = 100.0*volumes[i] / spec['max_vol']
        res['volumes'] = volumes
        if lsmc == True:
            lsmc_dispatch = analytics.vectorDouble()
            lsmc_price = analytics.GasStorageLSMCPricer.price(lsmc_dispatch, valdate, spec_new,model,
                                                     fwd_curve, bidask['none']['volumes'], 
                                                    bidask['none']['multiplier'], 
                                                    transaction_volume, transaction_cost, 
                                                      pathgenerator_param, 50)
            lsmc_volumes = [spec['start_volume']]
            for x in lsmc_dispatch:
                last = lsmc_volumes[-1]
                lsmc_volumes.append((last+x))
            for i in range(len(lsmc_volumes)):
                lsmc_volumes[i] = 100.0*lsmc_volumes[i] / spec['max_vol']
            res['lsmc_price'] = lsmc_price
            res['lsmc_dispatch'] = lsmc_dispatch
            res['lsmc_volumes'] = lsmc_volumes
            
        result[key] = res

if True: #volume_dependent_injection_cost_dispatch.value:
    plot_injection_cost(injections)

    #result2 = {}
    result = {}
    compute_injection_intrinsic(result, injections, True)

    #print(result['case 1']['intrinsic_price'])
    #print(result['none']['intrinsic_price'])
    plot_dispatch(result, 'injection_cost')
    

#### Price projection volume dependent injection costs

In [ ]:
def get_injection_costs(max_injection_cost):
    
    injections = {'flat': {'injection_cost_volumes': [0, 250000.0*MWh_to_m3_factor],
                    'injection_cost': [max_injection_cost,max_injection_cost]},
            'case 1': {'injection_cost_volumes': [0, 250000.0*MWh_to_m3_factor],
                    'injection_cost':[0.0,max_injection_cost]},
            'case 2': {'injection_cost_volumes': [0, 0.3*250000.0*MWh_to_m3_factor,0.31*250000.0*MWh_to_m3_factor, 250000.0*MWh_to_m3_factor],
                    'injection_cost': [0.5*max_injection_cost, 0.5*max_injection_cost, 1*max_injection_cost, 1.0*max_injection_cost]}
           }
    return injections

if volume_dependent_injection_cost_projection.value:
    max_injection_costs = pl.frange(0.0,4.0, 0.1250)
    prices_intrinsic={'flat':[], 'case 1' : [], 'case 2': []}
    prices_lsmc = {'flat':[], 'case 1' : [], 'case 2': []}
    for x in max_injection_costs:
        injection_costs = get_injection_costs(x)
        result={}
        compute_injection_intrinsic(result,injection_costs, True)
        for key in result.keys():
            prices_intrinsic[key].append(result[key]['intrinsic_price'])
            prices_lsmc[key].append(result[key]['lsmc_price'])

    rcParams['figure.figsize'] = 11.5, 4
    plt.figure()
    plt.subplot(1,2,1)
    for key, value in prices_intrinsic.items():
        plt.plot(max_injection_costs, value, '-x', label=key)
    plt.xlabel('maximal injection cost')
    plt.ylabel('value')
    plt.legend()
    plt.subplot(1,2,2)
    for key, value in prices_lsmc.items():
        plt.plot(max_injection_costs, value, '-x', label=key)
    plt.xlabel('maximal injection cost')
    plt.ylabel('value')
    plt.legend()
    plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
    plt.savefig('injection_cost_proj.png', dpi = 600)

### Injection/Withdrawal Rates

#### Projection w.r.t. volume dependent injection and withdrawal rates

In [ ]:
def project_spec(keys, percentages):
    result = []
    for perc in percentages:
        spec_new = copy.deepcopy(specs[selected_spec.value])
        for key in keys:
            if isinstance(spec_new[key], list):
                for i in range(len(spec_new[key])):
                    spec_new[key][i] *= perc
            else:
                spec_new[key] *=perc
        result.append(create_spec(spec_new))
    return result

percentages = pl.frange(0.8,1.5, 0.0125)

if volume_independent_injection_rate_projection.value:
    specs_proj = project_spec(['injection_rate'], percentages)
    specs_proj = project_spec(['withdrawal_rate', 'injection_rate'], percentages)

    intrinsic_prices = []
    lsmc_prices = []
    volume_points = 150
    num_injections = 100
    for spec_new in specs_proj:
        dispatch = analytics.vectorDouble()
        intrinsic_prices.append(analytics.GasStorageIntrinsicPricer.priceIntrinsic(dispatch, valdate, spec_new,
                                                         fwd_curve,  volume_points, num_injections, 
                                                         bidask['none']['volumes'], 
                                                        bidask['none']['multiplier'], 
                                                        transaction_volume, transaction_cost))
        lsmc_dispatch = analytics.vectorDouble()
        lsmc_prices.append(analytics.GasStorageLSMCPricer.price(lsmc_dispatch, valdate, spec_new,model,
                                                         fwd_curve, bidask['none']['volumes'], 
                                                        bidask['none']['multiplier'], 
                                                        transaction_volume, transaction_cost, 
                                                          pathgenerator_param, 50))


    plt.figure()
    plot_perc = []
    for x in percentages:
        plot_perc.append(100*x)
    plt.plot(plot_perc, intrinsic_prices,'-x', label='intrinsic')
    plt.plot(plot_perc, lsmc_prices,'-x', label='lsmc')
    plt.xlabel('maximum withdrawal/injection [% of original]')
    plt.ylabel('value')
    plt.legend()

In [ ]:
if volume_dependent_injection_rate_projection.value:
    injection_rates_proj = pl.frange( 0, 2500.0*MWh_to_m3_factor, 1250.0*MWh_to_m3_factor/20.0)
    injection_volumes = [0, 250000.0*MWh_to_m3_factor]
    lsmc_prices = []
    intrinsic_prices = []
    for x in injection_rates_proj:
        spec_new = copy.deepcopy(specs[selected_spec.value])
        spec_new['injection_rate_volumes'] = injection_volumes
        spec_new['injection_rate'] = [ 2500.0*MWh_to_m3_factor, x]
        gasstorage = create_spec(spec_new)
        intrinsic_prices.append(analytics.GasStorageIntrinsicPricer.priceIntrinsic(dispatch, valdate, gasstorage,
                                                         fwd_curve,  volume_points, num_injections, 
                                                         bidask['none']['volumes'], 
                                                        bidask['none']['multiplier'], 
                                                        transaction_volume, transaction_cost))
        lsmc_prices.append(analytics.GasStorageLSMCPricer.price(dispatch, valdate, gasstorage,model,
                                                         fwd_curve, bidask['none']['volumes'], 
                                                        bidask['none']['multiplier'], 
                                                        transaction_volume, transaction_cost, 
                                                          pathgenerator_param, 50))
    plt.figure()
    perc = []
    for x in injection_rates_proj:
        perc.append(100.0*x/(2500.0*MWh_to_m3_factor))
    plt.plot(perc, intrinsic_prices, '-x',label='intrinsic')
    plt.plot(perc, lsmc_prices, '-x' , label='lsmc')
    plt.xlabel('minmum injection rate (% of maximum injection at volume 0)')
    plt.ylabel('value')

In [ ]:
lsmc = True

if selected_spec_new.value:
    spec_list = [selected_spec.value, selected_spec_new.value]
    result={}
    for x in spec_list:
        res = {}
        specification = create_spec(specs[x])
        dispatch = analytics.vectorDouble()
        price = analytics.GasStorageIntrinsicPricer.priceIntrinsic(dispatch, valdate, specification,
                                                     fwd_curve,  volume_points, num_injections, 
                                                      bidask['none']['volumes'], 
                                                    bidask['none']['multiplier'], 
                                                    transaction_volume, transaction_cost)
        res['intrinsic_price'] = price
        print('intrinsic_price: ' + str(price))
        res['dispatch'] = dispatch
        volumes = [spec['start_volume']]
        max_vol = 1
        if 'max_vol' in spec.keys():
            max_vol = spec['max_vol']
        else:
            max_vol = spec['operation_details'][0]['max_vol']
        for y in dispatch:
            last = volumes[-1]
            volumes.append((last+y))
        for i in range(len(volumes)):
            volumes[i] = 100.0*volumes[i] / max_vol
        res['volumes'] = volumes
        if lsmc == True:
            lsmc_dispatch = analytics.vectorDouble()
            lsmc_price = analytics.GasStorageLSMCPricer.price(lsmc_dispatch, valdate, specification, model, fwd_curve, 
                                            bidask['none']['volumes'], 
                                            bidask['none']['multiplier'], 
                                           transaction_volume, transaction_cost, 
                                           pathgenerator_param, 50)
            volumes = [spec['start_volume']]
            for y in lsmc_dispatch:
                last = volumes[-1]
                volumes.append((last+y))
            for i in range(len(volumes)):
                volumes[i] = 100.0*volumes[i] / max_vol
            print('lsmc_price: ' + str(lsmc_price))
            res['lsmc_price'] = lsmc_price
            res['lsmc_dispatch'] = lsmc_dispatch
            res['lsmc_volumes'] = volumes
            name = x
        if x ==  'rivacon':
            name='volume dependent'
        else:
            if x == 'rivacon_injection_nonvolumedependent':
                name = 'constant'
        result[name] = res
    plot_dispatch(result, 'allegro')


In [ ]:
dispatch = analytics.vectorDouble()
res ={}
price = analytics.GasStorageIntrinsicPricer.priceIntrinsic(dispatch, valdate, gasstorage,
                                             fwd_curve,  volume_points, num_injections, 
                                              bidask['none']['volumes'], 
                                            bidask['none']['multiplier'], 
                                            transaction_volume, transaction_cost)
res['intrinsic_price'] = price
res['dispatch'] = dispatch
volumes = [spec['start_volume']]
max_vol = 1
if 'max_vol' in spec.keys():
    max_vol = spec['max_vol']
else:
    max_vol = spec['operation_details'][0]['max_vol']
for y in dispatch:
    last = volumes[-1]
    volumes.append((last+y))
for i in range(len(volumes)):
    volumes[i] = 100.0*volumes[i] / max_vol
    res['volumes'] = volumes
if lsmc == True:
    lsmc_dispatch = analytics.vectorDouble()
    lsmc_price = analytics.GasStorageLSMCPricer.price(lsmc_dispatch, valdate, gasstorage, model, fwd_curve, 
                                    bidask['none']['volumes'], 
                                    bidask['none']['multiplier'], 
                                   transaction_volume, transaction_cost, 
                                   pathgenerator_param, 50)
    volumes = [spec['start_volume']]
    for y in lsmc_dispatch:
        last = volumes[-1]
        volumes.append((last+y))
    for i in range(len(volumes)):
        volumes[i] = 100.0*volumes[i] / max_vol
    res['lsmc_price'] = lsmc_price
    res['lsmc_dispatch'] = lsmc_dispatch
    res['lsmc_volumes'] = volumes
    
def plot_dispatch_intrinsic_vs_lsmc(result):
    plt.figure()
    plt.plot(result['dispatch'],'-x', label = 'intrinsic')
    plt.plot(result['lsmc_dispatch'],'-x', label = 'lsmc')
        #plt.plot(value['lsmc_dispatch'],'-o', label = key)
    plt.xlabel('t (days)')
    plt.ylabel('injection/withdrawal')
    plt.legend(loc='upper right', prop={'size':10})
    plt.figure()
    plt.plot(result['volumes'],'-x', label = 'intrinsic')
    plt.plot(result['lsmc_volumes'],'-x', label = 'lsmc')
        #plt.result(value['lsmc_dispatch'],'-o', label = key)
    plt.xlabel('t (days)')
    plt.ylabel('volume')
    plt.legend(loc='upper right', prop={'size':10})

plot_dispatch_intrinsic_vs_lsmc(res)
print(res['lsmc_price'])